# 원, 삼각형, 사각형 구분하기

In [1]:
!unzip -qq handwriting_shape.zip -d handwriting_shape

In [2]:
!unzip -qq hard_handwriting_shape.zip -d hard_handwriting_shape

### 1. CNN 모델

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터 셋 생성하기

In [5]:
train_datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = train_datagen.flow_from_directory(
    'handwriting_shape/train',
    target_size=(24, 24),
    batch_size=3,
    class_mode='categorical'
)

Found 45 images belonging to 3 classes.


In [6]:
!ls handwriting_shape/train

circle	rectangle  triangle


In [7]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    'handwriting_shape/test',
    target_size=(24, 24),
    batch_size=3,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


In [8]:
train_generator.labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2], dtype=int32)

In [9]:
train_generator.filenames[0]

'circle/circle001.png'

### 모델 정의/설정/학습

In [10]:
model = Sequential([
                    Conv2D(32, (3, 3), activation='relu', input_shape=(24, 24, 3)),
                    Conv2D(64, (3, 3), activation='relu'),
                    MaxPooling2D(pool_size=(2, 2)),
                    Flatten(),
                    Dense(128, activation='relu'),
                    Dense(3, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               819328    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
__________________________________________________

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 모델 학습
- 첫번째 인자: 훈련데이터셋 제공하는 제너레이터
- steps_per_epoch: 한 epoch에 사용한 스텝 수 지정
- epochs: 학습 반복 횟수
- validation_data: 검증데이터셋 제공하는 제너레이터
- validation_steps: 한 epoch 종료 시마다 검증할 때 사용되는 검증 스텝 수 지정

In [12]:
model.fit_generator(
    train_generator,
    steps_per_epoch=15,
    epochs=50,
    validation_data=test_generator,
    validation_steps=5
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
15/15 [==============================] - 2s 54ms/step - loss: 1.6247 - accuracy: 0.5373 - val_loss: 0.8169 - val_accuracy: 0.6667
Epoch 2/50
15/15 [==============================] - 0s 22ms/step - loss: 0.5652 - accuracy: 0.7945 - val_loss: 0.2033 - val_accuracy: 0.9333
Epoch 3/50
15/15 [==============================] - 0s 23ms/step - loss: 0.0782 - accuracy: 0.9924 - val_loss: 0.0816 - val_accuracy: 1.0000
Epoch 4/50
15/15 [==============================] - 0s 23ms/step - loss: 0.0230 - accuracy: 1.0000 - val_loss: 0.0740 - val_accuracy: 0.9333
Epoch 5/50
15/15 [==============================] - 0s 23ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0659 - val_accuracy: 1.0000
Epoch 6/50
15/15 [==============================] - 0s 22ms/step - loss: 8.7783e-04 - accuracy: 1.0000 - val_loss: 0.0518 - val_accuracy: 1.0000
Epoch 7/50
15/15 [==============================] - 0s 23ms/step - loss: 5.5387e-04 - accuracy: 1.0000 - val_loss: 0.0490 - val_accuracy: 1.0000
Epoch

In [13]:
print('--Evaluation--')
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

--Evaluation--
accuracy: 100.00%


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
